In [1]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

/Users/jcastillo/anaconda/envs/Python36/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# NLTK Documentation http://www.nltk.org/howto/sentiment.html

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [6]:
n_instances = 1000
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)

(1000, 1000)

In [9]:
train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

In [10]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
len(unigram_feats)

83

In [11]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [13]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.8
F-measure [obj]: 0.8
F-measure [subj]: 0.8
Precision [obj]: 0.8
Precision [subj]: 0.8
Recall [obj]: 0.8
Recall [subj]: 0.8


# VADER-Sentiment-Analysis

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [7]:
sentences = ["VADER is smart, handsome, and funny.",      # positive sentence example
            "VADER is not smart, handsome, nor funny.",   # negation sentence example
            "VADER is smart, handsome, and funny!",       # punctuation emphasis handled correctly (sentiment intensity adjusted)
            "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
            "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
            "VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
            "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!",# booster words & punctuation make this close to ceiling for score
            "The book was good.",                                     # positive sentence
            "The book was kind of good.",                 # qualified positive sentence is handled correctly (intensity adjusted)
            "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
            "At least it isn't a horrible book.",         # negated negative sentence with contraction
            "Make sure you :) or :D today!",              # emoticons handled
            "Today SUX!",                                 # negative slang with capitalization emphasis
            "Today only kinda sux! But I'll get by, lol"  # mixed sentiment example with slang and constrastive conjunction "but"
             ]

for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<50} {}".format(sentence, str(vs)))

VADER is smart, handsome, and funny.-------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is not smart, handsome, nor funny.---------- {'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.7424}
VADER is smart, handsome, and funny!-------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is very smart, handsome, and funny.--------- {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545}
VADER is VERY SMART, handsome, and FUNNY.--------- {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227}
VADER is VERY SMART, handsome, and FUNNY!!!------- {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!! {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}
The book was good.-------------------------------- {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}
The book was kind of good.------------------------ {'neg': 0.0, 'neu': 0.657, 'pos': 0.343, 'comp

# Survey Data

In [3]:
cat Surveys.csv | head -n 5






cat: stdout: Broken pipe


In [4]:
import pandas as pd
import numpy as np

In [5]:
sData = pd.read_csv('Surveys.csv')
del sData['response_ID']
sData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8211091 entries, 0 to 8211090
Data columns (total 4 columns):
user#                  int64
survey_ID              object
date_user_responded    object
user_text_entry        object
dtypes: int64(1), object(3)
memory usage: 250.6+ MB


In [45]:
print (sData.iloc[0])

user#                                           31731264
survey_ID                                    advisor_nps
date_user_responded            2017-03-18 00:00:00 +0000
user_text_entry        I can see my balance at any time.
Name: 0, dtype: object


In [46]:
sData['date_user_responded'] = sData['date_user_responded'].astype('datetime64[ns]')

ValueError: Error parsing datetime string "<null>" at position 0

In [6]:
sData[40:45]

,user#,survey_ID,date_user_responded,user_text_entry
40,16507109,advisor_nps,2017-03-18 00:00:00 +0000,Enter outside investments manually without lin...
41,22558058,memsite,2017-03-18 00:00:00 +0000,Nothing.
42,22558058,memsite,2017-03-18 00:00:00 +0000,NaN
43,23428892,advisor_nps,2017-03-18 00:00:00 +0000,very well explained
44,23428892,advisor_nps,2017-03-18 00:00:00 +0000,it is very easy to understand


In [7]:
sData = sData.drop(sData[sData['user_text_entry'].isnull()].index)
sData[40:45]

,user#,survey_ID,date_user_responded,user_text_entry
40,16507109,advisor_nps,2017-03-18 00:00:00 +0000,Enter outside investments manually without lin...
41,22558058,memsite,2017-03-18 00:00:00 +0000,Nothing.
43,23428892,advisor_nps,2017-03-18 00:00:00 +0000,very well explained
44,23428892,advisor_nps,2017-03-18 00:00:00 +0000,it is very easy to understand
45,23428892,advisor_nps,2017-03-18 00:00:00 +0000,nothing


In [ ]:
df = pd.DataFrame(columns=['compound'])
for index, row in sData.iterrows():
    vs = analyzer.polarity_scores(row['user_text_entry'])
#   print("{:-<50} {}".format(row['user_text_entry'], str(vs)))
#   print (index, str(row['user#']) + ' ' + str(vs['compound']) + ':' + row['user_text_entry'])
    df.loc[index] = vs['compound']
    if (index % 100000 == 0):
        if (index > 0):
            print (index)
            df.to_csv('compounds_' + str(index) + '.csv')

100000
200000
300000
400000
500000


In [ ]:
surveyDF = pd.concat(sData, df)